Here I am just coding the model, you can copy the other modules like dataset creation and training from the 3dCNN code [here](https://github.com/khetansarvesh/video/blob/main/action_recognision/3DCNN.ipynb)!!

In [ ]:
class CNN_RNN_Model(nn.Module):
    def __init__(self):
        super(CNN_RNN_Model, self).__init__()

        self.conv = nn.Sequential(
                                    nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=0), nn.BatchNorm2d(32, momentum=0.01),nn.ReLU(inplace=True),
                                    nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0), nn.BatchNorm2d(64, momentum=0.01), nn.ReLU(inplace=True),
                                    nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=0), nn.BatchNorm2d(128, momentum=0.01), nn.ReLU(inplace=True),
                                    nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=0), nn.BatchNorm2d(256, momentum=0.01), nn.ReLU(inplace=True),
                                )


        self.conv1_outshape = conv2D_output_size((256, 342), (0,0), (5,5), (2,2))
        self.conv2_outshape = conv2D_output_size(self.conv1_outshape, (0,0), (3,3), (2,2))
        self.conv3_outshape = conv2D_output_size(self.conv2_outshape, (0,0), (3,3), (2,2))
        self.conv4_outshape = conv2D_output_size(self.conv3_outshape, (0,0), (3,3), (2,2))

        self.fnn1 = nn.Sequential(
                                      nn.Linear(256 * self.conv4_outshape[0] * self.conv4_outshape[1], 1024), nn.ReLU(),
                                      nn.Linear(1024, 768), nn.ReLU(),
                                      nn.Dropout(0.3),
                                      nn.Linear(768, 101)
                                  )


        self.LSTM = nn.LSTM( input_size=512, hidden_size=512, num_layers=3, batch_first=True)


        self.ffn2 = nn.Sequential(
                                  nn.Linear(512, 256), nn.ReLU(),
                                  nn.Linear(256, 101)
                                  )




    def forward(self, x_3d):
        cnn_embed_seq = []

        # iterating over each frame in the video
        for t in range(x_3d.size(1)):

            # CNNs
            x = self.conv(x_3d[:, t, :, :, :])

            # Flattening
            x = x.view(x.size(0), -1)

            # FC layers
            x = self.fnn1(x)
            cnn_embed_seq.append(x)

        # swap time and sample dim such that (sample dim, time dim, CNN latent dim)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1) # shape => (batch, time_step, input_size)

        # passing through LSTM Layer
        self.LSTM.flatten_parameters()
        lstm_output, (h_n, h_c) = self.LSTM(cnn_embed_seq, None)  # shape_lstm_output => (batch, time_step, output_size), shape_h_n => (n_layers, batch, hidden_size), shape_h_c => (n_layers, batch, hidden_size)

        # passing through FFNN Layer
        final_output = self.fnn2(lstm_output[:, -1, :])   # choose x at the last time step

        return final_output
